From this notebook two types of lists are made:
Non-stammable list of words (nost) which contain such part of speech tags:
- Saikļi
- Prievārdi
- Izsauksmes vārdus
- Partikulas

Stammable list of words (st) which contain such part of speech tags:
- Apstākļa vārds
- Darbības vārds
- Divdabis
- Lietvārds
- Skaitļa vārds
- Īpašības vārds
- Vietniekvārds
- Īpašvārds

The dataset from which exception wordlists are created is Tēzaurs (Spektors, Andrejs; et al.). The data file can be found in tezaurs_2022_tei-xml.zip/tezaurs_2022_tei.xml and downloaded here: https://repository.clarin.lv/repository/xmlui/handle/20.500.12574/66

Extracting entities with their part of speech tags

In [29]:
from pathlib import Path
def getFilter(path="parquet", prefix="", match="", ignore="", suffix="parquet"):
    """
    Get list of files given path, prefix, match, ignore, suffix"""
    p = Path(path)
    if ignore:
        return [f for f in p.iterdir() if f.is_file() and f.name.startswith(prefix) and match in f.name and not ignore in f.name and f.name.endswith(suffix)]
    return [f for f in p.iterdir() if f.is_file() and f.name.startswith(prefix) and match in f.name and f.name.endswith(suffix)]

In [30]:
import xml.etree.ElementTree as ET

files = getFilter("./", suffix=".xml")
text = files[0].read_text(encoding="utf-8")
root = ET.fromstring(text)

entities, x = set(), set()
for entity in root[1][:-1]:
    for form in entity.findall("./form"):
        orth = form.find("./orth").text
        noflecw = form.findall(".//*[@type='Nelokāms vārds']")
        prop_name = form.findall(".//*[@type='Īpašvārda veids']")
        pos = form.findall(".//*[@type='Vārdšķira']")

        if noflecw:
            p = "Nelokāms vārds"
        elif prop_name:
            if orth[-1] in ["o", "ā", "ē", "o", "u", "ū", "f"]:
                p = "Nelokāms vārds"
            else:
                p = "Īpašvārds"
        elif len(pos) > 0:
            p = pos[0].text            # The first is taken since the rest are either pos of following word or smth else 
        else:
            p = ""
        entities.add((orth, p))
len(entities)

395554

In [31]:
wordlist, abbr = set(), set()
with_symbol = set() # fallo-, -cm, Hewlett-Packard
for words, pos in entities:
    spl_words = words.split()
    for w in spl_words:
        w = w.lower()
        if w[-1] == ".":
            # abbr.add(w[:-1]) VECAIS
            wordlist.add((w[:-1], "Saīsinājums"))
        else:
            if w.isalpha():
                if len(spl_words) > 1:
                    wordlist.add((w, ""))
                else:
                    wordlist.add((w, pos))
                    
            else:
                with_symbol.add(w)
len(wordlist)

372496

Creating and stemming a set of nouns (including toponyms and excluding words in a foreign language) in 2nd declination plural form

In [32]:
# Consonant alternation
# Since masculine words in 6th and words in 5th deklination in plural genitive and all words in 2nd plural stems in LatvianStemmer different than in other inflections
# (e.g.: /blūze, ļaudis/ stem("...žu") is "...ž" not "...d" or "...z"; /ballīte, kase  stem("...šu") is "...š" not "...t" or "...s")
# such stemmed words must be added to VCC wordlist.
# Note: words in 6th declination are taken only if they were labeled so (since inflection -s is also for masculine words in 1st declination, there are no rules that
# could destinguish these declinations). 2nd and 5th declination were taken from wordlist if words ended with -is and -e, then manually reviewed. 
# Words (including toponyms and excluding words in a foreign language) in 2nd or 6th and 5th declination (words that end with -is, -e and -s) ¡that consists of VCC!
vcc_cc_is = ['semissis', 'missis', 'pertussis', 'hunnis', 'kunnis', 'minnis', 'nannis', 'ninnis', 'perennis', 'pienspannis', 'pinnis', 'pusvannis', 'rennis', 'sonnis', 'spannis', 'spinnis', 'spunnis', 'sviestspannis', 'tennis', 'tunnis', 'vannis', 'vinnis', 'ķinnis', 'ķunnis']
vcc_c_is = ['bullītis', 'caurrūdzis', 'cittautietis', 'garradzis', 'immerītis', 'kannaraudzis', 'kannasraudzis', 'kannuraudzis', 'kodolledlauzis', 'lollītis', 'lullītis', 'melldirsietis', 'melluzis', 'mellītis', 'mellūzis', 'mugurradzis', 'piparrausis', 'plakannadzis', 'pupulellītis', 'pussakritis', 'pussavītis', 'pusskābītis', 'pussprādzis', 'pussprīdis', 'rullītis', 'sausserdis', 'saussviedis', 'saussērdis', 'savvalnietis', 'sjerraleonietis', 'skrullastis', 'skrullītis', 'spannītis', 'sproggalvītis', 'spunnītis', 'tullūzis', 'turritis', 'vannūzis', 'viennadzis', 'villkārsis', 'villnesis', 'villītis', 'vīteņsausserdis', 'zemmilzis', 'ziemmitis', 'šļemmozis']
vcc_cc_e = ['asse', 'atte', 'garenšotte', 'gazette', 'jeunesse', 'karlshuette', 'kresse', 'laisse', 'lamette', 'latte', 'misse', 'mousse', 'natte', 'nozze', 'otte', 'paulette', 'promesse', 'rainette', 'recepisse', 'roulette', 'sedde', 'zisse', 'šotte']
vcc_c_e = ['annasbirze','annonse','annuite','annuitāte','annupīte','annāliste','appute','aspergilloze','asslodze','attestate','atturlīste','attēlkarte','baddzegoze','baddzeguse','baddzeguze','baddzegūze','bakkaste','ballenštete','ballīte','barrete','bellatrikse','bertiellioze','bezziede','bollate','bremmelīte','buļļupīte','celluloze','ceļmallapīte','ciemmaize','cirroze','cittautiete','debessbrūte','deggāze','dielektrollze','donavvērte','dālderroze','eksterritorialitāte','ellīte','enneāde','ezarroze','ezerroze','fibrillolīze','fibrilloģenēze','finnoze','finnsnēse','follete','galliete', 'hallande','hellebarde','hellēniste','hilodonelloze','himmelšpeize','hunniete','illegalitāte','irreliģiozitāte','irreversibilitāte','irritābilitāte','izzude','johanngeorgenštate','kallaste','kannīte','karrebeksminde','kastellabate','kastellamonte','kazzemenīte','kontrollente','korrektrise','kreimmaize','krejummaize','krellainīte','krellīte','kroppenštete','krsitalloze','krējummaize','kukaiņappute','kummīte','lagglāze','lappuse','lenneštate','limmate','lollande','mammicīte','mammucīte','mammīte','mannoze','mannozidoze','mazzante','mellenīte','melleņaudze','mellrihštate','mellupīte','mellužīte','mellīte','memmīte','merveilleuse','meļļupīte','miesassardze','millerioze','millroze','minnerštate','miļļupīte','mollase','mollese','motorolleriste','nabassaite','nulladrese','oberramštate','pamattonalitāte','pamattēze','pantalonnade','papildappute','papillomatoze','pašappute','piespieddzese','pijkannīte','pinnīte','pollakikoproze','pommade','programmbāze','programmizstrāde','programmkomponente','pulloroze','pussekunde','pusslodze','pusspunde','pussvēte','pīpkannīte','pļavvālodze','ralliste','rennede','rennerode','ritterhauze','rokassaite','rokassprādze','ruļļaste','rīpellāde','sannēse','sausserde','sirreāliste','sjerraleoniete','snieggāze','sottise','staļļupīte','stellīte','sudrabbisīte','suffraģiste','superreāliste','svešappute','tellese','tiņņupīte','tonnjūdze','trihinelloze','trummete','tullīte','tuvappute','tālappute','ummerštate','urrāpatriote','vakarreize','vallensfelze','vannīte','varroatoze','varroze','veccēzīte','vellmāte','viddienpuse','villabāte','violončelliste','viņammāte','zemessardze','zemesstērste','zieppuķīte','zonnevalde','čelliste','ēbermannštate','ķemmlāde','ļukkurpīte','šellefte','šiemmāte','šimmāte']
vcc_c_s = ['diennakts', 'mēnessnakts']
# Conversion rules to convert in plural form (genetive) and stem them. Only such rules that will change a words after stemming are used. 
cc_alt = {'nn': "ņņ", 'ss': "šš", "tt": "šš", "dd": "žž", "zz": "žž"}
c_end = {'d': 'ž', 's': 'š', 't': 'š', 'z': 'ž'}

In [33]:
vcc_dekl_st_plur = set()
for w in vcc_cc_is:
    if w[-4:-2] in cc_alt:
        vcc_dekl_st_plur.add(w[:-4] + cc_alt[w[-4:-2]])
for w in vcc_cc_e:
    if w[-3:-1] in cc_alt:
        vcc_dekl_st_plur.add(w[:-3] + cc_alt[w[-3:-1]])
len(vcc_dekl_st_plur)

45

In [34]:
for w in vcc_c_is:
    if w[-4] != w[-3] and w[-3] in c_end:
        vcc_dekl_st_plur.add(w[:-3] + c_end[w[-3]])
for w in vcc_c_e:
    if w[-3] != w[-2] and w[-2] in c_end:
        vcc_dekl_st_plur.add(w[:-2] + c_end[w[-2]])
vcc_dekl_st_plur.add(vcc_c_s[0][:-2]+"š")   # 'diennakts'
vcc_dekl_st_plur.add(vcc_c_s[1][:-2]+"š")   # 'mēnessnakts'
len(vcc_dekl_st_plur)

269

Stemming wordlist words and adding stemmed words in 2nd declination plural form. Collecting words that ends with VCC

In [35]:
from LatvianStemmer import stem

consonants = ["b", "c", "č", "d", "f", "g", "ģ", "h", "j", "k", "ķ", "l", "ļ", "m", "n", "ņ", "p", "r", "s", "š", "t", "v", "z", "ž"]
svowels = ["a", "e", "i", "o", "u"]

st_pos = ['Lietvārds', 'Īpašības vārds', 'Skaitļa vārds', "Vietniekvārds", 'Īpašvārds', 'Darbības vārds', 'Divdabis']

end_vcc = set()
nost_wordlist = set()
st_wordlist = set([stem(w) for w in vcc_dekl_st_plur])

for w, p in wordlist:
    # Finding words that ends with VCC (bizness, ņamm)
    for v in svowels:
        for c in consonants:
            if v+c+c == w[-3:]:
                end_vcc.add(w)
                
    # Stemming wordlist
    if p in st_pos:
        st_wordlist.add(stem(w))
    else:
        nost_wordlist.add(w)


# Words where last two letters of stem are ņņ in stem() are stemmed incorrectily – "ņn". So to correct this mistake such words are replaced with ending "ņņ".
st_wordlist.remove("peņn")
st_wordlist.remove("ņuņn")
st_wordlist.add("peņņ")
st_wordlist.add("ņuņņ")

len(st_wordlist), len(nost_wordlist), len(end_vcc)

(113541, 243239, 954)

Adding words that ends with VCC. Adding prefixes and suffixes to VCC words.

In [10]:
# Suffixes that starts with consonant and could create VCC (with words which stem ends with the same letter).
# Such suffixes are only "niek" 

# suffix_pos =  ["Lietvārds", "Īpašības vārds", "Darbības vārds"]
# suf_st_wordlist = set()

# for w, p in st_wordlist:
#     if p in suffix_pos:
#         if w[-1] == "n":
#             suf_st_wordlist.add((w+"niek", p))
# len(suf_st_wordlist)

In [11]:
# # Prefixes without those who ends with vowel.
# # since no word starts with two consonants, combining them with such prefixes will not make VCC 
# prefix = ["aiz", "ap", "at", "bez", "caur", "iz", "pār", "pēc", "pret", "uz"]
# prefix_pos =  ["Apstākļa vārds", "Darbības vārds", "Divdabis", "Lietvārds", "Īpašības vārds"]

# st_wordlist.update(end_vcc)

# pref_st_wordlist = set()
# for w, p in st_wordlist:
#     if p in prefix_pos:
#         for pref in prefix:
#             if pref[-1] == w[0]:
#                 pref_st_wordlist.add(pref+w)
# len(pref_st_wordlist)

In [36]:
st_vcc = set()
for w in st_wordlist:
    for v in svowels:
        for c in consonants:
            if v+c+c in w:
                st_vcc.add(w)
len(st_vcc)

2740

In [20]:
#  Compound words are removed since all of them are related with chemistry or medicine 
# comp_w = ['kommerc', 'gross', 'milli', 'izza', 'hippo', 'all', 'allo', 'atto', 'fall', 'falli', 'cillo', 'fallo', 'ferr', 'ferri', 'ferro', 'fillo', 'psammo', 'pupill', 'pupillo', 'stanni', 'stanno']

#  Words with symbol are those that contained a symbol (except "."). Some symbols like '"' are removed and words in parenthesis are removed (since such words are already in other lists).
# w_with_symbol = ['Hewlett-Packard', 'Novell', 'Quattro', 'Tessa', 'Adsel-Luttershof', 'Alt-Sallensee', 'Apencelle-Auserrodene', 'Apencelle-Innerrodene', 'Apencelles-Auserrodenes', 'Apencelles-Innerrodenes', 'Bagge-Assieten', 'Bagge-Assiten', 'Basku-Navarras', 'Bārle-Nassau', 'Cella-Melisa', 'Demmen-Pastorat', 'Fillingene-Šveningene', 'Frank-Sessau', 'Gallas-Somālijas', 'Gazenpot-Paddernskaja', 'Gross-Dueppeln', 'Gross-Dursuppen', 'Gross-Lassen', 'Gross-Roennen', 'Gross-Sessau', 'Gross-Sussei', 'Grumma-Gržimailo', 'Hellebeka-Ālsgārde', 'Hornbeka-Dronningmelle', 'Huballi-Dhāravāda', "J'accuse", 'Katvaru-Vannišu', 'Kilingi-Nemme', 'Klaustāle-Cellerfelde', 'Klein-Aisswicken', 'Klein-Dueppeln', 'Klein-Dursuppen', 'Klein-Kalleten', 'Klein-Roennen', 'Klein-Sessau', 'Klein-Strutteln', 'Klein-Sussei', 'Korneti-Peļļi', 'Krons-Roennen', 'Krussat-Drogen', 'Lubben-Oschen', 'Lubbert-Renzen', 'Masa-Karrāra', 'Mullutu-Sūrlahts', 'Neu-Sessau', 'Neu-Warriben', 'Paddern-Hasenpoht', 'Pallas-Illestunturi', 'Pallas-Ounastunturi', 'Rankas-Leļļu', 'Skaržiska-Kamenna', 'Skaržisko-Kamenna', 'Tasch-Padderen', 'Taš-Paddernskaja', 'Tels-Paddern', 'Vanne-Eikele', 'Viššī-Broda', 'Wittenheim-Sussei', "all'Jonio", "all'ottava", "all'unisono", 'allii-populina', 'anagallis-aquatica', 'chill-out', 'dell’Emīlija', "drum'n'bass", 'e-bizness', 'gamma-tokoferols', 'glass-paper', 'half-penny', 'haut-dessus', 'haut-taille', 'ilang-ilangeļļa', 'karmaņņiks-tjehņiks', 'krak-process', 'lurgi-metalls', 'mell-elksnis', 'mūzik-halls', 'nell’Emīlija', 'penna-marina', 'penny-a-liner', 'permo-triass', 'pis-aller', 'pot-pourri', 'rosae-pimpinellifolia', "sull'Ada", "sull'Arno", 'sull’Oglio', 'vaccinii-uliginosi', 'vallu', 'Ķemeru-Pussēkļa', 'Šteibaha-Hallenberga', 'šerri-brendi', 'šerri-brendijs', 'šerri-koblers']
# w_with_symbol = set([w.lower() for w in w_with_symbol])

# nost_wordlist.update(w_with_symbol)
# nost_wordlist.update(abbr)

nost_vcc = set()
for w in nost_wordlist:
    for v in svowels:
        for c in consonants:
            if v+c+c in w:
                nost_vcc.add(w)

nost_vcc.update(end_vcc)

len(nost_vcc)

8252

In [ ]:
nost_vcc.remove("arr")
nost_vcc.remove("irr")
nost_vcc.remove("att")
nost_vcc.remove("essi")

# st_vcc.remove("tikk")
st_vcc.remove("mann")
st_vcc.remove("viņņ")
st_vcc.remove("arr")
st_vcc.remove("att")

Adding compound words, company names and words with simbol to full list of VCC. Compound words and words with symbol were found after filtering them out at the beginning and searching for those that have VCC. These words do not combine with prefixes (**This is omitted**)

In [ ]:

nost_vcc = set()
for w in nost_wordlist | w_with_symbol:
    for v in svowels:
        for c in consonants:
            if v+c+c in w:
                nost_vcc.add(w)

nost_vcc.update(end_vcc)

len(nost_vcc)

In [28]:
# remove_prefix ||PREF||
# Removes prefix
def remove_prefix(w, rm_all=True):
    for p in ['atpakaļ', 'līdzās', 'priekš', 'pretim', 'līdzās', 'pret', 'līdz', 'caur', 'vis', 'pie', 'pār', 'aiz', 'pus', 'uz', 'sa', 'pa', 'no', 'ne', 'jā', 'iz', 'ie', 'at', 'ap']:
        # Takes each prefix from "w_p_e.pref" list and checks if word starts with it. Measures prefix length (taken from the list) and checks that many first letters in the word; if they match, prefix is removed
        if w[:len(p)] == p:
            nopref_w = w[len(p):]

            # the minimal length of word without prefix is 2, because the input words can be stemmed. e.g., "neas" from word "neass"
            if len(nopref_w)<=1:
                return (w, "")

            #  if property rm_all is set to False, word without a prefix and removed prefix is returned. If not, the rest of the prefixes will be removed from word and returned
            if not rm_all:
                return (nopref_w, p)
            
            # To remove the remaining prefixes, remove_prefix() function is called. Both its values are returned, but before the second value returned prefix is added
            new_w, new_p = remove_prefix(nopref_w)
            return (new_w, p + new_p)
    # If word dont have any prefix, word with nothing is returned
    return (w, "")

In [287]:
st_vcc = set([remove_prefix(w)[0] for w in st_vcc])
nost_vcc = set([remove_prefix(w)[0] for w in nost_vcc])

In [242]:
# piem., netto
vcc_in_pref_w_st = set()
for w in st_wordlist:
    for p in ['atpakaļ', 'līdzās', 'priekš', 'pretim', 'līdzās', 'pret', 'līdz', 'caur', 'vis', 'pie', 'pār', 'aiz', 'pus', 'uz', 'sa', 'pa', 'no', 'ne', 'jā', 'iz', 'ie', 'at', 'ap']:
        if w[:len(p)] == p:
            w = w[len(p):]
            if w[:1] == w[1:2] and w[1:2] and w[1:2] in consonants:
                vcc_in_pref_w_st.add(w)
vcc_in_pref_w_nost = set()
for w in nost_wordlist:
    for p in ['atpakaļ', 'līdzās', 'priekš', 'pretim', 'līdzās', 'pret', 'līdz', 'caur', 'vis', 'pie', 'pār', 'aiz', 'pus', 'uz', 'sa', 'pa', 'no', 'ne', 'jā', 'iz', 'ie', 'at', 'ap']:
        if w[:len(p)] == p:
            w = w[len(p):]
            if w[:1] == w[1:2] and w[1:2] and w[1:2] in consonants:
                vcc_in_pref_w_nost.add(w)

In [ ]:
st_adj_adv_with_s = set()
for w, p in wordlist:
    if p in ["Apstākļa vārds", "Īpašības vārds"]:
        if w[0] == "s":
            st_adj_adv_with_s.add(stem(w))
len(st_adj_adv_with_s)

1185

In [19]:
with open('./../GitHub_Faili/lang_resources/vcc_wordlist.py', 'w', encoding="utf8") as f:
    f.write('st_vcc = ' + str(st_vcc))
    f.write('\nnost_vcc = ' + str(nost_vcc))
    f.write('\nst_adjectives_with_s = ' + str(st_adj_adv_with_s))

# Prefixes, suffixes and word endings

In [17]:
# ig -> īg
nost_ig, st_ig = set(), set()
for w, p in wordlist:
    if stem(w)[-2:] == "ig":
        st_ig.add(w)

for w in st_wordlist:
    if w[-2:] == "ig":
        st_ig.add(w)
len(nost_ig), len(st_ig)
# 20, 45

(0, 452)

In [ ]:
for w in nost_wordlist:
    if stem(w)[-2:] == "ig" and not stem(w) in st_wordlist:
        print(w)

In [ ]:
# nal -> nāl
x = set()
for w, p in wordlist:
    if stem(w)[-3:] == "nal":

    # # if p == "Darbības vārds":
    #     for c in consonants:
    #             # if w[-4:] == c+"ej"+e:
    #             if w[-3:] == c+"et":
                    x.add(w)
len(x)

In [50]:
# For converting prefix pec to pēc. There is no pec_st because it only consists of 1 word – "pec" ||PREF||
# pec = set()
# for w in nost_wordlist: 
#     if w[:3] == "pec":
#         pec.add(w)
# for i in ['pec', 'pecten', 'pectinata', 'pectinatae', 'pectinatum', 'pectinatus', 'pectinicornis', 'pectinidae', 'pectorale', 'pectoralis', 'pectore', 'pectunculus', 'pectus', 'peculium', 'pecunia', 'pecus']:
#     pec.remove(i)
# len(pec)

12

In [58]:
# For converting prefix ja to jā if vowel follows the prefix. "jau..." is excluded, since ||PREF||
vowels = ["a", "ā", "e", "ē", "i", "ī", "o", "ū"]
ja_v_nost, ja_v_st = set(), set()
for w in st_wordlist:
    for v in vowels:
        if w[:3] == "ja"+v:
            ja_v_st.add(w)
for w in nost_wordlist:
    for v in vowels:    
        if w[:3] == "ja"+v:
            ja_v_nost.add(w)

ja_v_nost.remove("jaera")
len(ja_v_nost), len(ja_v_st)
# After this, some words are moved to stemmed list and removed. Final list: 
# ja_v_nost = {'jaegel', 'jaera', 'jaina', 'jao'}
# ja_v_st = {'jaan', 'jaaņ', 'jaic', 'jaik', 'jaikstin', 'jaikstiņ', 'jaikst', 'jail', 'jain', 'jaipan', 'jair', 'jairin', 'jairiņ', 'jairīt', 'jaiv', 'jaolof'}

(16, 5)

In [237]:
# Lai atrastu vārdus, kas sākas ar itkā priedēkli, kam seko itkā nepareizs preidēklis
# piem paISums, izBESīt
pref_bes_st, pref_bes_nost = set(), set()
word = "lids"
for w in st_wordlist:
    w, pr = remove_prefix(w, True)
    if word == w[:len(word)] and pr:
        pref_bes_st.add(pr+w)
for w in nost_wordlist:
    w, pr = remove_prefix(w, True)
    if word == w[:len(word)] and pr:
        pref_bes_nost.add(pr+w)
len(pref_bes_nost), len(pref_bes_st)

(0, 0)

In [63]:
# Converting prefixes that ends with "z" ||PREF||

# For converting prefix lidz and lids to līdz 
lidz_st, lidz_nost = set(), set()
for w in st_wordlist:
    if w[:4] == "lidz":
        lidz_st.add(w)
for w in nost_wordlist:
    if w[:4] == "lidz":
        lidz_nost.add(w)

lids_st, lids_nost = set(), set()
for w in st_wordlist:
    if w[:4] == "lids":
        lids_st.add(w)
for w in nost_wordlist:
    if w[:4] == "lids":
        lids_nost.add(w)

lidz_nost.remove('lidz')
lidz_nost.remove('lidzbarka') #Pilsēta Polijā (jau ir iekļauta pie _st)
print("lidz_st, lidz, lids_st, lids: ", len(lidz_st), len(lidz_nost), len(lids_st), len(lids_nost))

# Converting ais, bes, is, us
ais_nost, ais_st, bes_nost, bes_st, is_nost, is_st, us_nost, us_st = set(), set(), set(), set(), set(), set(), set(), set()
for w in st_wordlist:
    if len(w) > 2:
        if w[:2] == "is":
            is_st.add(w)
        if w[:2] == "us":
            us_st.add(w)
        if len(w) > 3:
            if w[:3] == "ais":
                ais_st.add(w)
            # if w[:3] == "bes":
            #     bes_st.add(w)
for w in nost_wordlist:
    if len(w) > 2:
        if w[:2] == "is":
            is_nost.add(w)
        if w[:2] == "us":
            us_nost.add(w)
        if len(w) > 3:
            if w[:3] == "ais":
                ais_nost.add(w)
            # if w[:3] == "bes":
            #     bes.add(w)
# bes.remove("bessianum")

for i in ['islandes', 'isabellina', 'isoetopsida', 'isochrysidaceae', 'isogeneratophyceae', 'isabellinus', 'isariopsis', 'ischnura', 'isothecium', 'ischaemum', 'isoachlya', 'ischias', 'islaya', 'isospora', 'islayensis', 'isophylla', 'ischnopsyllus', 'isvintenu']
    is_nost.remove(i)
for i in ['usque', 'ustilaginales', 'usitatissimum', 'usnea', 'ustulina', 'usuriensis', 'usneoides', 'usneaceae']:
    us_nost.remove(i)
print("ais, ais_st, bes, bes_st, is, is_st, us, us_st:", len(ais_nost), len(ais_st), len(bes_nost), len(bes_st), len(is_nost), len(is_st), len(us_nost), len(us_st))

# Updating with old Latvian characters ("zt", "zd", "zp" if they are not in prefix; "ee" and "tš", "tž"):
for w in ais_nost | ais_st | bes_nost | bes_st | is_nost | is_st | us_nost | us_st | lidz_nost | lidz_st | lids_nost | lids_st | ais_nost | ais_st | bes_nost | bes_st | is_nost | is_st | us_nost | us_st:
    if "ie" in w:
        print(w, w.replace("ie", "ee".upper()), " |ie| ", w.replace("ie", "ee"))

    if "č" in w:
        print(w, w.replace("č", "tš".upper()), " |tš| ", w.replace("č", "tš"))
    if "č" in w:
        print(w, w.replace("č", "tž".upper()), " |tž| ", w.replace("č", "tž"))
    
    if "h" in w:
        print(w, w.replace("h", "ch".upper()), " |h| ", w.replace("h", "ch"))

    if "st" in w:
        print(w, w.replace("st", "zt".upper()), " |st| ", w.replace("st", "zt"))
    if "sd" in w:
        print(w, w.replace("sd", "zd".upper()), " |sd| ", w.replace("sd", "zd"))
    if "sp" in w:
        print(w, w.replace("sp", "zp".upper()), " |sp| ", w.replace("sp", "zp"))
ja_v_st.update(['jaikzt', 'jaikztin', 'jaikztiņ'])
lids_st.update('lidzpalv', 'lidzpēj', 'lidsabeedrib', 'lidsabeedrīb')
ais_nost.update(['aisteeši', 'aisterneeki', 'aisītees'])
ais_st.update(['beserētees', 'besteneeki', 'aischils'])
bes_st.update(['besazeed'])
bes_nost.update('bestenchof')
is_nost.update(['ischāks', 'isichija', 'ischēmiskais', 'isichazma', 'ischakzaji', 'ischēmiskā', 'istabazpuķe', 'ispleikzt', 'isreedneeki', 'istalsneeši', 'istabneeks', 'isleenas', 'isnaudeeši', 'istreeši', 'istabasleetas', 'istabazdurvis', 'iskolaztrels', 'ismaēleeši', 'iskreeši', 'istabazduris', 'istabaspreekša', 'istabazpriekša', 'islandeešu', 'istžavkaķ', 'istšavkaķ', 'isatženko', 'isatšenko', 'istaštžīt', 'istaštšīt'])
is_st.update(['islandeet', 'islakristin', 'istabpreekš', 'istableet', 'isleen', 'isteej', 'isatženk', 'isatšenk', 'islāmšachr', 'istrascch', 'isfachān', 'islachij', 'isperich', 'isdžechisar'])
us_nost.update(['ustjeeši', 'ustroneeši', 'ustabneeku', 'uszp', 'usteeši', 'ustabpreekša', 'usmeneeki', 'uskatševas', 'uskatževas'])
us_st.update(['ustjkatšk', 'ustjkatžk', 'uskatšov', 'uskatžov', 'ustiluch', 'uscchwald', 'uscchwalden'])

(15, 11, 20, 30, 149, 69, 83, 50)

### Endings

In [51]:
# c_ib
ib_st, ib_nost = set(), set()
for w in nost_wordlist:
    w, pref = remove_prefix(w, True)
    st_w = stem(w)
    if len(st_w)>2 and st_w[-3] in consonants and st_w[-2:] == "ib":
        ib_nost.add(w)
for w in st_wordlist:
    w, pref = remove_prefix(w, True)
    if len(w)>2 and w[-3] in consonants and w[-2:] == "ib":
        ib_st.add(w)
len(ib_nost), len(ib_st),

(139, 48)

In [52]:
# ak
ak_nost, ak_st = set(), set()
for w in nost_wordlist:
    w, pref = remove_prefix(w, True)
    st_w = stem(w)
    if len(st_w) > 2:
        if st_w[-2:] == "ak":
            ak_nost.add(w)
for w in st_wordlist:
    w, pref = remove_prefix(w, True)
    # There is problem with "pasaka". After removing pref, w is "k" which does not end with "ak"
    if len(w) > 2:
        if w[-2:] == "ak":
            ak_st.add(w)
len(ak_nost), len(ak_st)

(1164, 381)

In [165]:
# c_sv_šan
short_v = ["a", "e", "i", "u"]
long_v = ["ā", "ē", "ī", "ū"]
vowels = {"a", "e", "i", "o", "u", "ā", "ē", "ī", "ū"}
c_sv_san_nost, c_sv_san_st = set(), set()
for w in nost_wordlist:
    w, pref = remove_prefix(w, True)
    st_w = stem(w)
    if len(st_w) > 5:
        if st_w[-5] in consonants and st_w[-4] in short_v and st_w[-3:] == "šan":
            c_sv_san_nost.add(w)

for w in st_wordlist:
    w, pref = remove_prefix(w, True)
    if len(w) > 5:
        if w[-5] in consonants and w[-4] in short_v and w[-3:] == "šan":
            c_sv_san_st.add(w)
# len(c_sv_san_st), len(c_sv_san_nost): 10; 26

I_conj = {'atģisties', 'pasākt', 'pamosties', 'nobeigties', 'iztikt', 'novilkt', 'triekties', 'pārģērbt', 'izgāzties', 'uzvilkt', 'šķiest', 'spēt', 'kalt', 'sanākt', 'izspiest', 'novelties', 'uzkārties', 'izlīst', 'pārvilkt', 'saprast', 'apklāt', 'atstiept', 'uzsnigt', 'mosties', 'iesvilties', 'sagrābt', 'sagrūt', 'palūgt', 'reibt', 'noklāt', 'uzbraukt', 'izklīst', 'pielikt', 'ievilkties', 'dūkt', 'apņemties', 'pastiept', 'izgriezt', 'atgriezties', 'riebties', 'sažūt', 'sadegt', 'apgrauzt', 'aizslēgties', 'sacept', 'ielaist', 'iekļauties', 'izslēgties', 'noliet', 'sakrāties', 'nosūkt', 'nopūsties', 'saspringt', 'ieslīgt', 'ieplūst', 'uzart', 'vilties', 'plaukt', 'nīst', 'noliekties', 'ieslēgties', 'mulst', 'noņemt', 'pielauzt', 'ieskriet', 'pārlauzt', 'nobrēkties', 'nosmieties', 'sapērt', 'ģērbt', 'pārciest', 'atzīties', 'sākties', 'pietikt', 'ienīst', 'nodzīt', 'izbeigties', 'nocirst', 'jozt', 'pamāt', 'pārspēt', 'savīt', 'kost', 'rakt', 'lieties', 'pūt', 'steigties', 'piesmelt', 'pāršķelt', 'nirt', 'vākt', 'nosvērties', 'liet', 'nokult', 'saplīst', 'izgrūst', 'apsvilt', 'iesēt', 'liegties', 'piezagties', 'pieliet', 'prast', 'nozvelties', 'aizvilkt', 'plūkt', 'pakārt', 'slābt', 'salīt', 'sarkt', 'kliegt', 'aizrīties', 'tiepties', 'ciesties', 'ielikties', 'nosaukt', 'justies', 'aizdrāzties', 'smakt', 'izsūkt', 'izlikties', 'uzvilkties', 'sajukt', 'nociesties', 'saglābt', 'iesaukt', 'pārkalst', 'izvest', 'apstāties', 'cept', 'blenzt', 'apmesties', 'lūgties', 'iedvest', 'lauzt', 'pieliekties', 'novest', 'piešķirt', 'atrast', 'paspēt', 'iegūt', 'beigties', 'izbeigt', 'atsiet', 'dīgt', 'piekāpties', 'nosmiet', 'uzziest', 'iekost', 'sazagt', 'aizkraut', 'izvelties', 'novīt', 'ievest', 'šļūkt', 'mīt', 'piesisties', 'brist', 'nodot', 'uzšūt', 'pasprukt', 'iebāzt', 'atvilkt', 'iegrauzties', 'nomīt', 'piesnigt', 'atklāt', 'sacensties', 'velties', 'sasprāgt', 'iesākt', 'gulties', 'sabīties', 'izkulties', 'pāraugt', 'iebraukt', 'apreibt', 'iešļūkt', 'izvilkt', 'pievērt', 'aizsniegt', 'pamest', 'izplaukt', 'nostiept', 'satikties', 'aizdzīt', 'norīt', 'pārplēst', 'aplēst', 'uzsākt', 'salt', 'nokrist', 'saprasties', 'slēpties', 'ietvert', 'izstiept', 'pagaist', 'velt', 'iesēsties', 'apkampties', 'sapazīties', 'uzdzīt', 'sagult', 'gaist', 'sanest', 'nākt', 'jaukt', 'pateikties', 'sasvīst', 'atgrūst', 'iejaukties', 'aizgriezt', 'aizskart', 'apcelt', 'gausties', 'izslēgt', 'nojukt', 'noreibt', 'atmaigt', 'ģērbties', 'nokalst', 'šūt', 'nozvelt', 'noslāpt', 'aizvērt', 'bēgt', 'braukt', 'sakāpt', 'pūst', 'pieņemties', 'stāties', 'nonākt', 'sakrāt', 'noņemties', 'aizbēgt', 'iedūkties', 'vērst', 'aizsaukt', 'pabērt', 'jāt', 'atkāpties', 'noliegt', 'izcelties', 'segt', 'cepties', 'klāt', 'apsvīst', 'iegulties', 'atpūsties', 'apmulst', 'iekarst', 'pazust', 'kalst', 'ķert', 'noģērbties', 'karst', 'ņirgt', 'iesūkties', 'piešļūkt', 'iekrauties', 'ieknābt', 'just', 'noģērbt', 'izdzīt', 'iezagties', 'pievērst', 'uznest', 'spert', 'izsvīst', 'stiept', 'burties', 'pirkt', 'nomesties', 'nākties', 'aizņemt', 'iepūt', 'uzkliegt', 'kopt', 'uzmesties', 'iesnausties', 'nogāzties', 'sakopties', 'saslēgt', 'pārtikt', 'samirkt', 'izzust', 'izlauzt', 'aiziet', 'padzert', 'pārplūst', 'piesmiet', 'atriebties', 'durties', 'saplēst', 'apgūt', 'apsvērt', 'sakulties', 'uzkrist', 'sariesties', 'ielauzties', 'ieiet', 'pieciest', 'piebērt', 'dzert', 'censties', 'satīt', 'aizmirst', 'nodūkt', 'art', 'nosviest', 'ienākt', 'ilgt', 'sist', 'snigt', 'sasnigt', 'rakties', 'sasalt', 'dzimt', 'pakļauties', 'ielūzt', 'iedot', 'piemest', 'izņemt', 'saplēsties', 'nobriest', 'notvert', 'cirpt', 'gāzties', 'aizrakt', 'pļaut', 'aiznesties', 'bāzt', 'izsmieties', 'spraust', 'atteikt', 'patriekt', 'piesarkt', 'apskaut', 'spriest', 'ierasties', 'sarukt', 'salipt', 'izsalt', 'nosēsties', 'piegrauzt', 'lēkt', 'nosvērt', 'apaut', 'rāt', 'birt', 'jaukties', 'izdurties', 'vērties', 'izvelt', 'piebraukt', 'svilt', 'nocirpt', 'izklāt', 'iedzīt', 'pievilt', 'izlaist', 'noaut', 'pārkraut', 'ieliet', 'iegrūst', 'noburt', 'sadurt', 'novelt', 'iekulties', 'izkrauties', 'pasniegt', 'atmest', 'sakraut', 'sviest', 'paust', 'sašūt', 'lemt', 'aizcirsties', 'mirkt', 'iziet', 'pieiet', 'padot', 'aizsniegties', 'ieplīst', 'raut', 'sasēsties', 'aizrauties', 'nosnaust', 'izšķiest', 'pārbēgt', 'noskūt', 'iešūt', 'iedurt', 'būt', 'izbērt', 'aizsprukt', 'saglaust', 'slīkt', 'satikt', 'ietīt', 'nodzēst', 'iespraukties', 'izsvērt', 'saiet', 'noplēst', 'uzsūkt', 'piesēst', 'sabārties', 'aut', 'savelt', 'nošķirt', 'iznirt', 'nozust', 'atlipt', 'sastrēgt', 'notīt', 'norēkties', 'noķert', 'pateikt', 'pievest', 'pāršļūkt', 'pieļaut', 'izlikt', 'pievilkt', 'kārt', 'iekārt', 'salaist', 'pieteikties', 'izpērties', 'aizlēkt', 'pārliet', 'izgrauzties', 'apspļaut', 'nolemt', 'pārtvert', 'satraukties', 'jukt', 'izsaukties', 'pārtapt', 'apvest', 'izšūt', 'izcirst', 'ieklāt', 'saplūkt', 'mirt', 'nobļaut', 'salauzt', 'vākties', 'noveikties', 'apspriest', 'svīst', 'pieslēgt', 'aizlikt', 'bērt', 'pieteikt', 'izdot', 'izsiet', 'pārbrist', 'pieklāties', 'izšļākt', 'aptīt', 'nopelt', 'pārlaist', 'piedzerties', 'pārraut', 'atgulties', 'izbēgt', 'ataugt', 'grauzt', 'izsviest', 'aizmigt', 'izbērties', 'izcelt', 'izlauzties', 'atlaist', 'pakārties', 'pārdurties', 'ierauties', 'ļauties', 'sabraukt', 'izkalt', 'sisties', 'nokaut', 'notriekt', 'sakliegt', 'aizvākt', 'sakopt', 'grimt', 'nokust', 'uzķerties', 'pārvest', 'iestiept', 'skūties', 'padzerties', 'kniebt', 'līst', 'izrauties', 'izbrist', 'nogrūst', 'iesist', 'uzlīt', 'krāpties', 'nopirkt', 'uzspēt', 'apjaust', 'piekliegt', 'noteikt', 'kampt', 'sacirst', 'noaugt', 'lēst', 'pagriezties', 'apšļākt', 'pasteigties', 'nokampt', 'pietrūkt', 'laist', 'lipt', 'nokost', 'izvērt', 'iedzert', 'pierast', 'rasties', 'sajust', 'zviegt', 'mēgt', 'piesist', 'apskaust', 'pienest', 'ienest', 'griezt', 'pazīt', 'atzīt', 'iepirkt', 'atspiesties', 'šaut', 'sarasties', 'piebilst', 'apģērbt', 'izlēkt', 'palikt', 'kaut', 'piekrist', 'uzgriezt', 'ietriekt', 'tērpt', 'šļākties', 'nopērties', 'novērst', 'uzdot', 'nosisties', 'plēst', 'sakarst', 'pieliekt', 'pārcelties', 'izlemt', 'ielīst', 'sakrist', 'atdot', 'saķert', 'aizsvīst', 'klīst', 'sabārt', 'izmesties', 'ņemties', 'ieskaut', 'pieskriet', 'nolikt', 'nosist', 'izkalst', 'nozagt', 'izdzert', 'atskārst', 'noplūst', 'aiznest', 'noplūkt', 'liekties', 'samulst', 'pārgāzt', 'ieklīst', 'noskriet', 'izkārt', 'pārkrist', 'izpaust', 'sajaukt', 'izstiepties', 'atstāt', 'saliet', 'izdoties', 'celt', 'aizcirst', 'sadzīt', 'pārvērsties', 'atvērt', 'grūst', 'ienirt', 'iegriezt', 'izliet', 'plīst', 'vilkties', 'piepūst', 'aizjozt', 'griezties', 'atnesties', 'aizklāt', 'uzaugt', 'izzagt', 'iespļaut', 'sagulties', 'ēst', 'bāzties', 'aizsnausties', 'piekopt', 'saģērbties', 'pārsēsties', 'spiest', 'izcept', 'izrakt', 'apkrāpt', 'iestumt', 'sastapt', 'raukt', 'doties', 'iesmelt', 'sakārt', 'apriebties', 'sasieties', 'savākties', 'piekrāpt', 'izsist', 'apzagt', 'augt', 'jēgt', 'ieaugt', 'pakāpties', 'brukt', 'atrauties', 'atcelt', 'piekļaut', 'izjaukt', 'izjukt', 'pārkost', 'sasniegt', 'dzīties', 'ievākties', 'sagriezt', 'sacirpt', 'līt', 'pārkarst', 'uzdoties', 'nomākt', 'palaist', 'rēkt', 'padzīt', 'sakauties', 'iegrauzt', 'riet', 'pielūgt', 'uztraukties', 'bīties', 'izglābt', 'aizbraukt', 'nobristies', 'pielīst', 'pagūt', 'iekraut', 'iztriekt', 'sasaukt', 'aizvērties', 'saraukt', 'atsist', 'pārkāpt', 'uzbriest', 'izbāzt', 'sasmelt', 'pārsvērt', 'uzveikt', 'lauzties', 'aizaugt', 'izspļaut', 'paciest', 'uznākt', 'uzsvērt', 'pieņemt', 'apiet', 'noriet', 'aizšūt', 'šķist', 'pārkalt', 'svērt', 'sakampties', 'paēst', 'apsveikt', 'lekties', 'plūst', 'noskūties', 'nolaist', 'izstāties', 'piesteigties', 'saņemties', 'aizkrist', 'piejaukt', 'iespēt', 'grābt', 'pārlikt', 'pārnākt', 'aizšļūkt', 'aizvilkties', 'noklīst', 'izteikt', 'izkust', 'pārcirst', 'izteikties', 'pieēst', 'uzraut', 'izdegt', 'pielēkt', 'sastapties', 'nospert', 'nosnausties', 'piesalt', 'iesniegt', 'piekalst', 'piepūsties', 'krāties', 'izjust', 'piesegties', 'iedegt', 'sakaut', 'nograuzt', 'sakalst', 'iesviest', 'atsteigties', 'pārbīties', 'uzdurties', 'pieaugt', 'sēt', 'sabāzt', 'pakampt', 'rast', 'sazagties', 'pieķert', 'piekalt', 'smieties', 'izvākties', 'sasaukties', 'pasaukt', 'iegrimt', 'iesprūst', 'sasiet', 'atvilkties', 'rīt', 'sagrauzt', 'ziest', 'noliekt', 'aizņemties', 'izskaust', 'strēbt', 'nolikties', 'triekt', 'izcepties', 'apjukt', 'durt', 'izmirkt', 'izsprukt', 'slēgt', 'krāpt', 'tapt', 'pārstāt', 'iespraust', 'glābties', 'aizslēgt', 'iekalt', 'tikties', 'nolauzt', 'aptvert', 'nolēkt', 'pagriezt', 'nošauties', 'atslēgties', 'izstumt', 'smiet', 'pārskriet', 'pārvelties', 'malties', 'vērsties', 'aizšķirt', 'pakļūt', 'nīkt', 'novākt', 'nest', 'migt', 'nodzerties', 'nonest', 'nosarkt', 'paveikt', 'aiztraukties', 'piesieties', 'apgriezties', 'pārdegt', 'atslābt', 'noslīkt', 'trūkt', 'atvest', 'iekaist', 'ievākt', 'panīkt', 'celties', 'zagties', 'izpērt', 'atsisties', 'nodegt', 'aizķert', 'uzcelt', 'piecelties', 'piekārt', 'savīst', 'iet', 'aizvest', 'saplūkties', 'uzmākties', 'iekniebt', 'uzklāt', 'ieilgt', 'malt', 'ķerties', 'ierieties', 'saukt', 'pārliekties', 'uzrasties', 'rauties', 'nokopt', 'uzmest', 'nokārt', 'pāršūt', 'panākt', 'sniegt', 'sēsties', 'skumt', 'pārģērbties', 'ielikt', 'smelt', 'izģērbties', 'vīt', 'izžūt', 'pārsniegt', 'izpārdot', 'iesaukties', 'atņemt', 'nobraukt', 'piedzimt', 'bļaut', 'lūzt', 'atgūt', 'izprast', 'izplīst', 'nosmelt', 'apsēsties', 'iemest', 'pacelties', 'izgrauzt', 'iesūkt', 'dot', 'atlikt', 'aizliegt', 'tīt', 'saraut', 'sabrēkt', 'atbraukt', 'sakult', 'saklāt', 'piešauties', 'savilkt', 'pārcensties', 'pieskaut', 'iegriezties', 'sakurt', 'aizklāties', 'nolaisties', 'noslēpt', 'ieņemt', 'aizspert', 'pārcelt', 'piesiet', 'sūkt', 'aplieties', 'uzkopt', 'iegāzt', 'uztvert', 'izgaist', 'nospļauties', 'izkrist', 'burt', 'atslēgt', 'nosalt', 'aizstāt', 'salikt', 'izraut', 'uztīt', 'izsmelt', 'noskaisties', 'glābt', 'stumt', 'apēst', 'piespļaut', 'ielūgt', 'apmākties', 'apsmiet', 'pārēsties', 'nočiept', 'noraut', 'šķirties', 'uzplīties', 'iesteigties', 'traukties', 'māt', 'atģist', 'atsperties', 'nogulties', 'apķert', 'nogāzt', 'dzēst', 'nogurt', 'sabeigt', 'glausties', 'sagraut', 'izgāzt', 'piemesties', 'izbrukt', 'ļaut', 'iemigt', 'izknābt', 'sajozt', 'atkrist', 'atrasties', 'attapties', 'izšķirties', 'kauties', 'paveikties', 'iznīkt', 'pārdurt', 'iekrist', 'kraut', 'piesūkties', 'nobrēkt', 'pielīt', 'lēkties', 'aplikt', 'izbīties', 'nokāpt', 'uzspiest', 'ietiepties', 'nesties', 'krist', 'pārtraukt', 'aizliet', 'pārsviest', 'pārvērst', 'plēsties', 'pārsaukt', 'notikt', 'iestāties', 'norauties', 'izglābties', 'piekāpt', 'slēgties', 'pārdot', 'piedot', 'atnest', 'pielipt', 'iztriekties', 'veikt', 'saņemt', 'tērpties', 'aiztikt', 'iebrukt', 'skriet', 'uzlauzt', 'sadurties', 'pienākt', 'apkampt', 'izliekties', 'izlaisties', 'kļūt', 'pērties', 'kāpt', 'aizbērt', 'pārtīt', 'sniegties', 'izvākt', 'nolīt', 'iekļaut', 'pērt', 'paņemt', 'salīst', 'piekaut', 'nogriezties', 'grūt', 'iedurties', 'samalt', 'uzkarst', 'pārlēkt', 'apģērbties', 'smelties', 'sašļukt', 'beigt', 'izkļūt', 'aizvākties', 'atbilst', 'nošļākt', 'knābt', 'atšķirties', 'noklāties', 'ievilkt', 'izcirpt', 'nodurt', 'sasist', 'nožūt', 'sabļaut', 'noprast', 'gūt', 'pelt', 'uzstāt', 'iebilst', 'mest', 'piestāt', 'ielēkt', 'bārt', 'sprukt', 'plūkties', 'skūt', 'paiet', 'apjēgt', 'novērsties', 'apliet', 'degt', 'piegriezties', 'pierīties', 'ciest', 'pārsist', 'iekurt', 'drāzties', 'izsaukt', 'saģērbt', 'noriebties', 'iecirst', 'uzsist', 'kult', 'piebāzt', 'nopērt', 'apcept', 'uzņemties', 'atšķirt', 'pārslēgt', 'ietilpt', 'vīst', 'novīst', 'likties', 'noknābt', 'izspraukties', 'pievērsties', 'aizķerties', 'saslimt', 'liekt', 'apsegt', 'diegt', 'sašķelt', 'sakampt', 'nogrimt', 'izdzēst', 'atsaukties', 'izaugt', 'iepazīt', 'apbraukt', 'iekļūt', 'izdrāzties', 'pielikties', 'izdurt', 'pāriet', 'uzkāpt', 'izkāpt', 'apsegties', 'atsākt', 'satraukt', 'mesties', 'izsēsties', 'sabrist', 'piesēsties', 'iesprukt', 'samest', 'nospiest', 'nojaust', 'svērties', 'padoties', 'iekāpt', 'apmest', 'sapūt', 'nojaukt', 'nošķelt', 'siet', 'apieties', 'pavilkt', 'uzsaukt', 'izspert', 'nosvilt', 'izkraut', 'patverties', 'uzmīt', 'pabeigties', 'nosūkties', 'slāpt', 'izplēst', 'sasirgt', 'rieties', 'pacelt', 'pieēsties', 'krāt', 'kust', 'stiepties', 'pasnaust', 'šķelt', 'vērt', 'posties', 'pārmest', 'nopļaut', 'pārnest', 'zvelt', 'iesvilt', 'pārmirkt', 'palaisties', 'ieskrieties', 'uzlikt', 'pārslēgties', 'žūt', 'aizsalt', 'šķelties', 'apkopt', 'pieķerties', 'čiept', 'izbraukt', 'klāties', 'nošaut', 'ņemt', 'savelties', 'atsākties', 'pārplīst', 'aizbļaut', 'uzstāties', 'aizsmakt', 'piekļūt', 'iepazīties', 'pārgrauzt', 'ietriekties', 'ieviest', 'pagrūst', 'iezviegties', 'piesaukt', 'vest', 'attiekties', 'sveikt', 'iepirkties', 'izšķirt', 'izšļākties', 'atraut', 'rāpties', 'nomirt', 'ieslēgt', 'gāzt', 'sašķelties', 'iejusties', 'graut', 'noplīst', 'iestrēgt', 'izsisties', 'nokristies', 'pieslēgties', 'izstrēbt', 'nošļukt', 'apsnigt', 'kulties', 'izsmiet', 'apdzīt', 'aizskriet', 'grauzties', 'atvērties', 'atļaut', 'sabrukt', 'teikt', 'atsaukt', 'izciest', 'ieteikt', 'nomest', 'veikties', 'iebēgt', 'izpirkt', 'sanīkt', 'pieklīst', 'zagt', 'sarauties', 'nopūt', 'pārsteigt', 'bārties', 'uzkost', 'pārliekt', 'skaust', 'atiet', 'atteikties', 'tvert', 'pielaist', 'slēpt', 'sprūst', 'atsieties', 'paļauties', 'iecelt', 'pārknābt', 'salēkt', 'noslēgt', 'atriebt', 'iespert', 'saaugt', 'iedegties', 'pārgriezt', 'iepatikties', 'uzvesties', 'pieveikt', 'spļaut', 'samesties', 'pievākt', 'nokliegties', 'nolīst', 'pieglausties', 'dzīt', 'snaust', 'briest', 'pārkrauties', 'lūgt', 'tverties', 'uzrauties', 'uzņemt', 'pārrauties', 'sasisties', 'steigt', 'uzsūkties', 'viest', 'apgriezt', 'izsniegt', 'apburt', 'apriet', 'iegāzties', 'piesegt', 'applūst', 'pārsmieties', 'sirgt', 'izģērbt', 'noslēpties', 'ievērt', 'sākt', 'likt', 'ievīt', 'riebt', 'sastingt', 'iebrist', 'aizdot', 'vilkt', 'nocelt', 'pabeigt', 'uztraukt', 'sviesties', 'nokļūt', 'aiztriekties', 'izdīgt', 'aizlauzt', 'piespriest', 'piegult', 'piekļauties', 'satvert', 'nošļūkt', 'ieraut', 'izkārties', 'uzplaukt', 'šļākt', 'apnikt', 'ierēkties', 'nolieties', 'tikt', 'ieriebt', 'savākt', 'iekliegties', 'izšļūkt', 'pārsalt', 'cirst', 'iekopt', 'glaust', 'izbāzties', 'liegt', 'samīt', 'aizlikties', 'iešauties', 'aprīt', 'iesākties', 'uziet', 'pieglaust', 'apjozt', 'nosmakt', 'noiet', 'pabraukt', 'nobrukt', 'skart', 'izkopt', 'brēkt', 'apspiest', 'aizsteigties', 'nošūt', 'nosegt', 'nosvīst', 'uzcept', 'uzbrukt', 'nostāties', 'iekrāt', 'uzcirsties', 'piespiest', 'iedrāzties', 'pārlīst', 'apcepties', 'patikt', 'zust', 'aizmest', 'izsteigties', 'nospriest', 'atļauties', 'izkrāpt', 'pārbraukt', 'atnākt', 'piešūt', 'nogriezt', 'nobīties', 'nozviegties', 'tiekties', 'šķirt', 'iebērt', 'izķert', 'piedzīt', 'mīties', 'noskumt', 'izmest', 'šļukt'}
for w in I_conj:
    if w[-2:] == "st":
        w = w[:-2]
    else:
        continue
    if len(w)>1 and w[-2] in consonants and w[-1] in short_v:
        w, pref = remove_prefix(w, True)
        c_sv_san_st.add(w+"šan")

len(c_sv_san_st), len(c_sv_san_nost)

(21, 26)

In [195]:
# v_taj
sv_taj_st, sv_taj_nost = set(), set()
for w in st_wordlist:
    if len(w)>3 and w[-4] in short_v and w[-3:] == "taj":
        w, pref = remove_prefix(w, True)
        sv_taj_st.add(w)
        
for w in nost_wordlist:
    w_st = stem(w)
    if len(w_st)>3 and w_st[-4] in short_v and w_st[-3:] == "taj":
        w, pref = remove_prefix(w, True)
        sv_taj_nost.add(w)
        

sv_tāj_st, sv_tāj_nost = set(), set()
for w in st_wordlist:
    if len(w)>3 and w[-4] in short_v and w[-3:] == "tāj":
        w, pref = remove_prefix(w, True)
        sv_tāj_st.add(w)

for w in nost_wordlist:
    w_st = stem(w)
    if len(w_st)>3 and w_st[-4] in short_v and w_st[-3:] == "tāj":
        w, pref = remove_prefix(w, True)
        sv_tāj_nost.add(w)

# There are no words in nost_wordlist that ends with lv + "tāj"
# There are no words in st_wordlist that ends with lv + "taj" (exc: 'botētaj', 'birzētaj', 'šreģistrētaj' which are written incorrectly)

len(sv_taj_st), len(sv_taj_nost), len(sv_tāj_st), len(sv_tāj_nost)

(10, 2, 3, 17)

In [37]:
with open('./../GitHub_Faili/lang_resources/w_prefixs_endings.py', 'w', encoding="utf8") as f:
    f.write('pref = ' + str(pref))
    f.write('\n'+'pref_exc_aiz = ' + str(pref_exc_aiz))
    f.write('\n'+'pref_exc_bez = ' + str(pref_exc_bez))
    f.write('\n'+'pref_exc_iz = ' + str(pref_exc_iz ))
    f.write('\n'+'pref_exc_uz = ' + str(pref_exc_uz ))


    f.write('\n'+'ib_exc_st = ' + str(ib_exc_st))
    f.write('\n'+'ib_exc_nost = ' + str(ib_exc_nost))

    f.write('\n'+'c_v_san = ' + str(c_v_san))
    
    f.write('\n'+'v_taj = ' + str(v_taj))
    f.write('\n'+'sv_taj = ' + str(sv_taj))

    # f.write('\n'+'c_v_t = ' + str(c_v_t))
    # f.write('\n'+'c_v_j = ' + str(c_v_j))


##### Letter conversion

In [66]:
# Removes prefix ||PREF||
def remove_prefix(w, rm_all=True):
    for p in ['atpakaļ', 'līdzās', 'priekš', 'pretim', 'līdzās', 'pret', 'līdz', 'caur', 'vis', 'pie', 'pār', 'aiz', 'pus', 'uz', 'sa', 'pa', 'no', 'ne', 'jā', 'iz', 'ie', 'at', 'ap']:
        # Takes each prefix from "w_p_e.pref" list and checks if word starts with it. Measures prefix length (taken from the list) and checks that many first letters in the word; if they match, prefix is removed
        if w[:len(p)] == p:
            nopref_w = w[len(p):]

            # the minimal length of word without prefix is 2, because the input words can be stemmed. e.g., "neas" from word "neass"
            if len(nopref_w)<=1:
                return (w, "")

            #  if property rm_all is set to False, word without a prefix and removed prefix is returned. If not, the rest of the prefixes will be removed from word and returned
            if not rm_all:
                return (nopref_w, p)
            
            # To remove the remaining prefixes, remove_prefix() function is called. Both its values are returned, but before the second value returned prefix is added
            new_w, new_p = remove_prefix(nopref_w)
            return (new_w, p + new_p)
    # If word dont have any prefix, word with nothing is returned
    return (w, "")

In [389]:
# ZT→ST, ZD→SD, ZP→SP ||PREF||
z_tdp_exc = {}
z_tdp_exc["zt_nost"], z_tdp_exc["zd_nost"], z_tdp_exc["zp_nost"] = set(), set(), set()
z_tdp_exc["zt_st"], z_tdp_exc["zd_st"], z_tdp_exc["zp_st"] = set(), set(), set()
z_tdp_dict_nost, z_tdp_dict_st = {}, {}
for w in nost_wordlist:
    w = remove_prefix(w, True)[0]
    if "zt" in w:
        z_tdp_exc["zt_nost"].add(w)
        if "st" in w or "sd" in w or "sp" in w:
            w_old = w.replace("st", "zt").replace("sd", "zd").replace("sp", "zp")
            z_tdp_dict_nost[w_old] = w
            z_tdp_exc["zt_nost"].add(w_old)
    if "zd" in w:
        z_tdp_exc["zd_nost"].add(w)
        if "st" in w or "sd" in w or "sp" in w:
            w_old = w.replace("st", "zt").replace("sd", "zd").replace("sp", "zp")
            z_tdp_dict_nost[w_old] = w
            z_tdp_exc["zd_nost"].add(w_old)
    if "zp" in w:
        z_tdp_exc["zp_nost"].add(w)
        if "st" in w or "sd" in w or "sp" in w:
            w_old = w.replace("st", "zt").replace("sd", "zd").replace("sp", "zp")
            z_tdp_dict_nost[w_old] = w
            z_tdp_exc["zp_nost"].add(w_old)
for w in st_wordlist:
    w = remove_prefix(w, True)[0]
    if "zt" in w:
        z_tdp_exc["zt_st"].add(w)
        if "st" in w or "sd" in w or "sp" in w:
            w_old = w.replace("st", "zt").replace("sd", "zd").replace("sp", "zp")
            z_tdp_dict_st[w_old] = w
            z_tdp_exc["zt_st"].add(w_old)
    if "zd" in w:
        z_tdp_exc["zd_st"].add(w)
        if "st" in w or "sd" in w or "sp" in w:
            w_old = w.replace("st", "zt").replace("sd", "zd").replace("sp", "zp")
            z_tdp_dict_st[w_old] = w
            z_tdp_exc["zd_st"].add(w_old)
    if "zp" in w:
        z_tdp_exc["zp_st"].add(w)
        if "st" in w or "sd" in w or "sp" in w:
            w_old = w.replace("st", "zt").replace("sd", "zd").replace("sp", "zp")
            z_tdp_dict_st[w_old] = w
            z_tdp_exc["zp_st"].add(w_old)
len(z_tdp_dict_nost), len(z_tdp_dict_st), len(z_tdp_exc["zt_st"]), len(z_tdp_exc["zd_st"]), len(z_tdp_exc["zp_st"]), len(z_tdp_exc["zt_nost"]), len(z_tdp_exc["zd_nost"]), len(z_tdp_exc["zp_nost"])

# Updating with old Latvian characters ("ch", "šan", "tāj", "īb", āk):
z_tdp_exc["zt_st"].update(["sovchoztechnikum", "termiņuzturēšan", "tāluztveršan", "šizteikšan", "radiouztveršan", "slimībizturig", "vibroizturib", "mitrumizturib", "karstumizturib", "usumizturib", "šuzticib", "lizturib", "sviedrizturib", "daudztautib", "sārmizturib", "triecienizturib", "bāztib", "vējizturib", "siltumizturib", "reizticib", "ilgizturib", "mazturib", "dilumizturib", "beztiesib", "termoizturib", "mazticib", "gaismizturib", "ūdensizturib", "ugunsizturib", "dzimumlīdztiesib", "slimībizturib", "karztumizturib"])
z_tdp_exc["zd_st"].update(["chrazdan", "kēzdivāšārchej", "dziedātajztrazd", "dziedatajztrazd", "ligzdotaj", "dziedātajstrazd", "dziedatajstrazd", "siltumizdalītaj", "siltumizdalataj", "šaizdegamib", "vīzdegunib", "datorizdevniecib", "bezdomib", "ztarpizdevniecib", "bezdūšib", "bezdarbib", "daudzdievib", "starpizdevniecib", "mazdūšib", "bezdievib"])
z_tdp_exc["zp_st"].update(["bezprātib", "daudzpusib", "bezpersonib", "daudzpavalstniecib", "bezpavalstniecib", "bezpavalztniecib", "bezpilsonib", "daudzpavalztniecib", "mazpārveidib", "bezpalīdzib"])
z_tdp_exc["zt_nost"].update(["ztch", "šuzturešana", "mitrumuzturētaji", "mitrumuzturetaji", "gurumizturiba", "aukstumizturiba", "reizticibā", "beztroksniba", "korozijizturiba", "detonācijizturiba", "mosfērizturiba", "gurumizturibas", "aukztumizturiba", "ilgizturiba"])
z_tdp_exc["zd_nost"].update(["vozduchs", "achuramazda", "pizdochen", "zduchāčs", "pizdochens", "mēdītajztrazds", "mēditajztrazds", "mēdītajstrazds", "mēditajstrazds", "ceļotajztrazds", "daudzdzemdētaja", "daudzdzemdetaja", "ceļotajstrazds", "grauzdētajs", "grauzdetajs", "ūzderibas", "bezdarbiba", "daudzdomiba", "datorizdevniecibas", "zvirgzdainiba", "bezdibeniba", "brazdavaks", "vairakuzdevumu"])
z_tdp_exc["zp_nost"].update(["autouzpildītajs", "autouzpilditajs", "bezpamatiba", "daudzpirkstiba", "mazprātiba", "daudzpirkztiba", "greizprātiba", "mazpilsoniba"])
z_tdp_exc["zt_nost"] = z_tdp_exc["zt_nost"].remove("briezt")
z_tdp_dict_st.update({"dziedātajztrazd": "dziedātājztrazd", "ztarpizdevniecib": "ztarpizdevniecīb", "karztumizturib": "karztumizturīb", "daudzpavalztniecib": "daudzpavalztniecīb"})
z_tdp_dict_nost.update({"mēdītajztrazds": "mēdītājztrazds", "ceļotajztrazds": "ceļotājztrazds", "bezpavalztniecib": "bezpavalztniecīb", "daudzpirkztiba": "daudzpirkztība", "aukztumizturiba": "aukztumizturība"})

In [399]:
tž_exc_st, tš_exc_st, tž_exc_nost, tš_exc_nost, tš_exc_st_pref, tž_exc_st_pref, tš_exc_nost_pref, tž_exc_nost_pref = set(), set(), set(), set(), set(), set(), set(), set()
for w in nost_wordlist:
    w, pref = remove_prefix(w, True)
    if pref[-1:] == "t" and w[0] == "š":
        tš_exc_nost_pref.add("t"+w)
    if pref[-1:] == "t" and w[0] == "ž":
        tž_exc_nost_pref.add("t"+w)
    if "tš" in w:
        tš_exc_nost.add(w)
    if "tž" in w:
        tž_exc_nost.add(w)
for w in st_wordlist:
    w, pref = remove_prefix(w, True)
    if pref[-1:] == "t" and w[0] == "š":
        tš_exc_st_pref.add("t"+w)
    if pref[-1:] == "t" and w[0] == "ž":
        tž_exc_st_pref.add("t"+w)
    if "tš" in w:
        tš_exc_st.add(w)
    if "tž" in w:
        tž_exc_st.add(w)
len(tž_exc_st), len(tš_exc_st), len(tž_exc_nost), len(tš_exc_nost)
tž_tš_exc = {}
tž_tš_exc["tž_st"] = tž_exc_st
tž_tš_exc["tš_st"] = tš_exc_st
tž_tš_exc["tž_nost"] = tž_exc_nost
tž_tš_exc["tš_nost"] = tš_exc_nost
tž_tš_exc["tž_nost_pref"] = tž_exc_nost_pref
tž_tš_exc["tž_st_pref"] = tž_exc_st_pref
tž_tš_exc["tš_nost_pref"] = tš_exc_nost_pref
tž_tš_exc["tš_st_pref"] = tš_exc_st_pref

In [273]:
nie_st, nie_nost = set(), set()
for w in nost_wordlist:
    if "nie" == w[:3]:
        nie_nost.add(w)
    
for w in st_wordlist:
    if "nie" == w[:3]:
        nie_st.add(w)
len(nie_nost), len(nie_st)

(126, 45)

In [38]:
diacritic_char = {"ā", "č", "ē", "ģ", "ī", "ķ", "ļ", "ņ", "š", "ū", "ž"}
ch_words_nost, no_ch_words_nost, ch_words_st, no_ch_words_st = set(), set(), set(), set()
for w in nost_wordlist:
    # for i in diacritic_char:
        # if not i in w:          # Removing latin words
        #     if not w[-2:] in ["ae", "ea", "ia"] and not w[-3:] in ["mum", "ius"]:  # Removing latin words
    if "ch" in w:
        ch_words_nost.add(w)
    else:
        no_ch_words_nost.add(w)
for w in st_wordlist:
    if "ch" in w:
        ch_words_st.add(w)
    else:
        no_ch_words_st.add(w)
        
ch_nost, ch_st = set(), set()
for w in ch_words_nost:
    if not w.replace("ch", "h") in no_ch_words_nost and not stem(w.replace("ch", "h")) in no_ch_words_st:
        ch_nost.add(w)

for w in ch_words_st:
    if not w.replace("ch", "h") in no_ch_words_st:
        ch_st.add(w)
        
# Updating with "īb", "āk", "šan", "tāj": (There were no occurances)
len(ch_nost), len(ch_st)

(1483, 170)

In [70]:
# pref_vcc_c_pref_nost, pref_vcc_c_pref_st = [], []
pref_vcc_w_nost, pref_vcc_w_st = [], []

for i in nost_wordlist:
    # for pr in ['atpakaļ', 'līdzās', 'priekš', 'pretim', 'pret', 'līdz', 'caur', 'vis', 'pie', 'pār', 'aiz', 'pus', 'uz', 'sa', 'pa', 'no', 'ne', 'jā', 'iz', 'ie', 'at', 'ap']:
        # if w.startswith(pr):
            # new_i = i[len(pr):]
            w, pr = remove_prefix(i)
            if w[:1] == pr[-1:] and w[:1] in consonants and pr[-2:-1] in ["a", "e", "i", "o", "u"]:
                pref_vcc_w_nost.append(w)
                # for ppr in ['atpakaļ', 'līdzās', 'priekš', 'pretim', 'pret', 'līdz', 'caur', 'vis', 'pie', 'pār', 'aiz', 'pus', 'uz', 'sa', 'pa', 'no', 'ne', 'jā', 'iz', 'ie', 'at', 'ap']:
                #     if new_i[1:].startswith(ppr):
                #         pref_vcc_c_pref_nost.append(new_i)
for i in st_wordlist:
    # for pr in ['atpakaļ', 'līdzās', 'priekš', 'pretim', 'pret', 'līdz', 'caur', 'vis', 'pie', 'pār', 'aiz', 'pus', 'uz', 'sa', 'pa', 'no', 'ne', 'jā', 'iz', 'ie', 'at', 'ap']:
        # if w.startswith(pr):
            # new_i = i[len(pr):]
            w, pr = remove_prefix(i)
            if w[:1] == pr[-1:] and w[:1] in consonants and pr[-2:-1] in ["a", "e", "i", "o", "u"]:
                pref_vcc_w_st.append(w)
                # for ppr in ['atpakaļ', 'līdzās', 'priekš', 'pretim', 'pret', 'līdz', 'caur', 'vis', 'pie', 'pār', 'aiz', 'pus', 'uz', 'sa', 'pa', 'no', 'ne', 'jā', 'iz', 'ie', 'at', 'ap']:
                #     if new_i[1:].startswith(ppr):
                #         pref_vcc_c_pref_st.append(new_i)


# len(pref_vcc_c_pref_nost), len(pref_vcc_c_pref_st),
len(pref_vcc_w_nost), len(pref_vcc_w_st)

(616, 297)